# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6c6cad7340>
├── 'a' --> tensor([[-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554]])
└── 'x' --> <FastTreeValue 0x7f6c6cad7880>
    └── 'c' --> tensor([[ 0.1390, -1.4721, -0.0973,  1.6220],
                        [-1.7386, -0.2560, -2.1643, -1.7560],
                        [ 0.9594,  2.1858,  2.0054,  1.0638]])

In [4]:
t.a

tensor([[-1.0805,  1.2278, -1.3890],
        [-0.5148,  1.3931, -0.6554]])

In [5]:
%timeit t.a

63 ns ± 0.0598 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6c6cad7340>
├── 'a' --> tensor([[-0.4530,  0.8161,  0.2386],
│                   [-0.7083,  0.9764,  0.2619]])
└── 'x' --> <FastTreeValue 0x7f6c6cad7880>
    └── 'c' --> tensor([[ 0.1390, -1.4721, -0.0973,  1.6220],
                        [-1.7386, -0.2560, -2.1643, -1.7560],
                        [ 0.9594,  2.1858,  2.0054,  1.0638]])

In [7]:
%timeit t.a = new_value

60.7 ns ± 0.0505 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0805,  1.2278, -1.3890],
               [-0.5148,  1.3931, -0.6554]]),
    x: Batch(
           c: tensor([[ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638]]),
       ),
)

In [10]:
b.a

tensor([[-1.0805,  1.2278, -1.3890],
        [-0.5148,  1.3931, -0.6554]])

In [11]:
%timeit b.a

61.4 ns ± 0.0548 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1416, -0.9578, -0.4399],
               [ 0.1160,  1.5457,  0.5105]]),
    x: Batch(
           c: tensor([[ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.152 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 54.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6bb7dedd60>
├── 'a' --> tensor([[[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]],
│           
│                   [[-1.0805,  1.2278, -1.3890],
│                    [-0.5148,  1.3931, -0.6554]]])
└── 'x' --> <FastTreeValue 0x7f6bb7dedb80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 27.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6bb7d9ff40>
├── 'a' --> tensor([[-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554],
│                   [-1.0805,  1.2278, -1.3890],
│                   [-0.5148,  1.3931, -0.6554]])
└── 'x' --> <FastTreeValue 0x7f6bbe9f18e0>
    └── 'c' --> tensor([[ 0.1390, -1.4721, -0.0973,  1.6220],
                        [-1.7386, -0.2560, -2.1643, -1.7560],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 33.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 75.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1390, -1.4721, -0.0973,  1.6220],
                       [-1.7386, -0.2560, -2.1643, -1.7560],
                       [ 0.9594,  2.1858,  2.0054,  1.0638]],
              
                      [[ 0.1390, -1.4721, -0.0973,  1.6220],
                       [-1.7386, -0.2560, -2.1643, -1.7560],
                       [ 0.9594,  2.1858,  2.0054,  1.0638]],
              
                      [[ 0.1390, -1.4721, -0.0973,  1.6220],
                       [-1.7386, -0.2560, -2.1643, -1.7560],
                       [ 0.9594,  2.1858,  2.0054,  1.0638]],
              
                      [[ 0.1390, -1.4721, -0.0973,  1.6220],
                       [-1.7386, -0.2560, -2.1643, -1.7560],
                       [ 0.9594,  2.1858,  2.0054,  1.0638]],
              
                      [[ 0.1390, -1.4721, -0.0973,  1.6220],
                       [-1.7386, -0.2560, -2.1643, -1.7560],
                       [ 0.9594,  2.1858,  2.0054,  1.0638]],

In [26]:
%timeit Batch.stack(batches)

77.2 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638],
                      [ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638],
                      [ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638],
                      [ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638],
                      [ 0.1390, -1.4721, -0.0973,  1.6220],
                      [-1.7386, -0.2560, -2.1643, -1.7560],
                      [ 0.9594,  2.1858,  2.0054,  1.0638],
                      [ 0.1390, -1.4721, -0.0973,  1.6220],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 230 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 2.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
